In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

In [7]:
# Load model and labels
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5) # 255/2=127.5
model.setInputMean((127.5, 127.5, 127.5)) # mobilenet => [-1,1]
model.setInputSwapRB(True) # auto convert BGR to RGB

classLabels = []
file_name = 'Labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    # print(classLabels)

In [113]:
# Function to detect objects in an image
def detect_objects_in_image(change):
    clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        uploaded_file = change['owner']
        image_name = list(uploaded_file.keys())[0]
        image_data = uploaded_file[image_name]['content']
        image = np.frombuffer(image_data, np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        
        # Perform object detection on the image
        ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
        
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Convert image to base64 encoding
        _, buffer = cv2.imencode('.jpg', image)
        img_str = base64.b64encode(buffer).decode('utf-8')
        
        # Display the name of the uploaded image
        display(widgets.Label(value=f"Uploaded Image: {image_name}"))
        
        # Display annotated image
        display(Image(data=buffer, format='jpg'))

# Function to detect objects in a video
def detect_objects_in_video(change):
    # clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        uploaded_file = change['owner']
        video_path = io.BytesIO(uploaded_file['content'])
        
        # Perform object detection on the video
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
            
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(frame, boxes, (255, 0, 0), 2)
                    cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            
            cv2.imshow('Object Detection in Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

# Function to detect objects in webcam
def detect_objects_in_webcam(change):
    clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
            
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(frame, boxes, (255, 0, 0), 2)
                    cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            
            cv2.imshow('Object Detection in Webcam', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

# Create buttons for image, video, and webcam
image_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Image',multiple=False)
video_button = widgets.FileUpload(accept='.mp4', description='Video',multiple=False)
webcam_button = widgets.Button(description='Webcam')

# Attach event handlers to the buttons
image_button.observe(detect_objects_in_image, names='value')
video_button.observe(detect_objects_in_video, names='value')
webcam_button.on_click(detect_objects_in_webcam)

# Display the buttons
display(image_button, video_button, webcam_button)

FileUpload(value=(), accept='.jpeg, .jpg, .png', description='Image')

FileUpload(value=(), accept='.mp4', description='Video')

Button(description='Webcam', style=ButtonStyle())

In [112]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import time

# Function to handle the selection of the toggle buttons
def handle_toggle_change(change):
    if change['new'] == 'Image':
        clear_output()
        display(image_upload_button, detect_button, output)
    elif change['new'] == 'Video':
        clear_output()
        print("Video selected")
        # Implement video related functionality here
    elif change['new'] == 'Webcam':
        clear_output()
        print("Webcam selected")
        # Implement webcam related functionality here

# Function to handle image upload
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Display loader
        with output:
            clear_output()
            display(loader)
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression
        
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content)
        
        # Display the image
        with output:
            clear_output()
            display(img_widget)

# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with output:
        clear_output()
        uploaded_files = image_upload_button.value
        if len(uploaded_files) > 0:
            uploaded_file = uploaded_files[0]  # Select the first file in the tuple
            image_content = uploaded_file['content']
            img_widget = widgets.Image(value=image_content)
            display(img_widget)


# Create toggle buttons
toggle_buttons = widgets.ToggleButtons(
    options=['Image'],
    description='Select:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Upload Image', 'Select Video', 'Use Webcam']
)

# Create image upload button
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image')

# Create detect button
detect_button = widgets.Button(description='Detect')
detect_button.on_click(handle_detect_button_click)

# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

# Create output widget
output = widgets.Output()

# Attach event handlers to the toggle buttons and image upload button
toggle_buttons.observe(handle_toggle_change, names='value')
image_upload_button.observe(handle_image_upload, names='value')

# Display the toggle buttons, image upload button, detect button, and output widget
display(widgets.VBox([toggle_buttons, image_upload_button, detect_button, output]))


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import time

# Function to handle image upload--------------
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Display loader
        # with output:
        #     clear_output()
        #     display(loader)
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.01)  # Sleep for 10 milliseconds for smooth progression
        
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content)
        
        # Display the image
        with output:
            clear_output()
            display(img_widget)

# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with output:
        clear_output()
        uploaded_files = image_upload_button.value
        if len(uploaded_files) > 0:
            uploaded_file = uploaded_files[0]  # Select the first file in the tuple
            image_content = uploaded_file['content']
            img_widget = widgets.Image(value=image_content)
            display(img_widget)

# Create image upload button
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image')

#Create label
image_label=widgets.HTML(
    value="<h2>Object Detection in IMAGE</h2>"
)

# Create detect button
detect_button = button = widgets.Button(
    description='Detect in Image',
    disabled=False,
    button_style='Success',
    tooltip='Detect Image',
    icon='image' # (FontAwesome names without the `fa-` prefix)
)
detect_button.on_click(handle_detect_button_click)

# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Imageeeee...',
    bar_style='success',
    style={'bar_color': 'red','width': '200px'},
    orientation='horizontal'
)


# Create output widget
output = widgets.Output()

# Create container widget for image-related components
image_tab_content = widgets.VBox([image_label,image_upload_button,loader, widgets.HBox([detect_button], layout=widgets.Layout(margin='10px 0px 0px 0px')), output])

# Attach event handlers to the image upload button
image_upload_button.observe(handle_image_upload, names='value')


# Function to handle VIDEO upload------------------
def handle_video_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Simulate progress with sleep
        for i in range(101):
            video_loader.value = i
            time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression
        
        # # Extract video content
        # video_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # # Display the uploaded video
        # video_widget = widgets.Video(value=video_content, format='mp4', width=640, height=480, autoplay=False)
        
        # Display the video
        # with video_output:
        #     clear_output()
        #     display(video_widget)

# Function to handle detect video button click
def handle_detect_video_button_click(b):
    # Display the uploaded video
    with video_output:
        clear_output()
        uploaded_files = video_upload_button.value
        if len(uploaded_files) > 0:
            uploaded_file = uploaded_files[0]  # Select the first file in the tuple
            video_content = uploaded_file['content']
            video_widget = widgets.Video(value=video_content, format='mp4', width=640, height=480, autoplay=False)
            display(video_widget)

# Create video upload button
video_upload_button = widgets.FileUpload(accept='.mp4', description='Upload Video')

#Create label
video_label=widgets.HTML(
    value="<h2>Object Detection in VIDEO</h2>"
)

# Create detect video button
detect_video_button = button = widgets.Button(
    description=' Detect in Video',
    disabled=False,
    button_style='Danger',
    tooltip='Detect Video',
    icon='video' # (FontAwesome names without the `fa-` prefix)
)
detect_video_button.on_click(handle_detect_video_button_click)

# Create loader for video
video_loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Videooooooo...',
    bar_style='danger',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

video_loader.description_style = {'width': '200px'}  # Set minimum width


# Create output widget for video
video_output = widgets.Output()

# Create container widget for video-related components
video_tab_content = widgets.VBox([video_label,video_upload_button, video_loader, widgets.HBox([detect_video_button], layout=widgets.Layout(margin='10px 0px 0px 0px')), video_output])

# Attach event handlers to the video upload button
video_upload_button.observe(handle_video_upload, names='value')

# Webcam Detection--------------------------------
# Create label for webcam detection
webcam_label = widgets.HTML(value="<h2>Webcam Detection</h2>")

# Create button to open webcam
open_webcam_button = widgets.Button(
    description="Open Webcam",
      button_style='Success',
    tooltip='Open Webcam',
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)

# Create button to close webcam
close_webcam_button = widgets.Button(
    description="Close Webcam",
    disabled=True,
    button_style='Danger',
    tooltip='Close Webcam',
    icon='close' # (FontAwesome names without the `fa-` prefix)
)

# Function to open webcam
def open_webcam(b):
    # Disable open button and enable close button
    open_webcam_button.disabled = True
    close_webcam_button.disabled = False
    
    # Create VideoCapture object for webcam
    cap = cv2.VideoCapture(0)
    
    # Function to display webcam feed
    def display_webcam():
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        webcam_output.clear_output(wait=True)
        with webcam_output:
            clear_output(wait=True)
            display(widgets.Image(value=cv2.imencode('.jpg', frame)[1].tobytes()))

    # Display webcam feed
    display_webcam()
    
    # Update webcam feed continuously
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        display_webcam()

       # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):  # Check if 'q' key is pressed
            break

    # Release VideoCapture object
    cap.release()
    # Close webcam output
    close_webcam(None)
    # Clear the output of open webcam button
    clear_output()

# Function to close webcam
def close_webcam(b):
    # Enable open button and disable close button
    open_webcam_button.disabled = False
    close_webcam_button.disabled = True
    
    # Clear the webcam output
    webcam_output.clear_output()
    # Stop the webcam feed
    display.stop()
    
# Attach event handlers to the buttons
open_webcam_button.on_click(open_webcam)
close_webcam_button.on_click(close_webcam)

# Create output widget for webcam feed
webcam_output = widgets.Output()

# Define a container for the webcam label, open webcam button, close webcam button, and webcam output
webcam_controls = widgets.VBox([webcam_label, widgets.HBox([open_webcam_button, close_webcam_button]), webcam_output])


# Create Tab widget---------------
tab_contents = ['Imageमा Detection', 'Videoमा Detection', 'Webcamमा Detection']
tabs = widgets.Tab()

tabs.children = [image_tab_content, video_tab_content, webcam_controls]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)

# Display the Tab widget
display(tabs)


In [8]:
#TESTING ONLY VIDEO PART DIRECT:success=+++++++++++++++++++++++

#after success: replacing with video_output part for future ref
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, HTML
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
import tempfile
import time
import os


# Load model and labels
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5) # 255/2=127.5
model.setInputMean((127.5, 127.5, 127.5)) # mobilenet => [-1,1]
model.setInputSwapRB(True) # auto convert BGR to RGB

classLabels = []
file_name = 'Labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')

# VIDEO DETECTION--------------------------------
# Define global variables
uploaded_video_content = None

# # Function to handle VIDEO upload
# def handle_video_upload(change):
#     global uploaded_video_content

#     if len(change['new']) > 0:
#         uploaded_file = change['new'][0]  # Access the first element of the tuple
#         uploaded_video_content = uploaded_file['content']
            
#         # Simulate progress with sleep
#         for i in range(101):
#             video_loader.value = i
#             time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression
                
#         # if uploaded_video_content:
#         #     print("Uploaded video content received.")  # Debug print
#         # else:
#         #     print("No valid video content available. Please upload a video.")

# # Function to handle detect video button click
# def handle_detect_video_button_click(b):
#     global uploaded_video_content

#     if uploaded_video_content:
#         # Save the video content to a temporary file
#         with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
#             tmp_file.write(uploaded_video_content)
#             video_path = tmp_file.name

#         # Create VideoCapture object for uploaded video
#         capt = cv2.VideoCapture(video_path)

#         #check if video opened correctly or not
#         if not capt.isOpened():
#             raise IOError("Cannot open Video") #replace with webcam for webcam detection
        
#         font_scale= 3
#         font= cv2.FONT_HERSHEY_PLAIN
        
#         while True:
#             ret, frame= capt.read()
           
#             ClassIndex, confidence, bbox = model.detect(frame,confThreshold=0.55) #0.5= 50%
        
#             print(ClassIndex)
#             if(len(ClassIndex)!=0):
#                     for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(),bbox): #all 3 variable in zip
#                         if(ClassInd <=80):
#                             cv2.rectangle(frame, boxes,(255,0,0),2) #red color rectangle adn text is green
#                             cv2.putText(frame, classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font, fontScale=font_scale, color=(0,255,0), thickness=3)
            
#             cv2.imshow("Object Detection in Video", frame)
        
#             if cv2.waitKey(2) & 0xff==ord('q'):
#                 break
        
#         capt.release()
#         cv2.destroyAllWindows()


# Define global variables
uploaded_video_content = None

# Global variable to keep track of annotated video file count
annotated_video_count = 1

# Function to handle VIDEO upload
def handle_video_upload(change):
    global uploaded_video_content
    global annotated_video_count

    if len(change['new']) > 0:
        uploaded_file = change['new'][0]  # Access the first element of the tuple
        print("Uploaded file:", uploaded_file)  # Debug print

        # Extract the content of the uploaded file
        uploaded_file_content = uploaded_file['content']
        if uploaded_file_content:
            uploaded_video_content = uploaded_file_content

            # Simulate progress with sleep
            for i in range(101):
                video_loader.value = i
                time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression

            print("Uploaded video content:", uploaded_video_content)  # Debug print

            # Increment the counter for the annotated video file
            annotated_video_count += 1

            # Reset the counter if it exceeds a certain threshold (optional)
            if annotated_video_count > 100:
                annotated_video_count = 1
                
        else:
            print("No valid video content available. Please upload a video.")


# Function to handle detect video button click and perform object detection
def handle_detect_video_button_click(change):
    global uploaded_video_content
    global annotated_video_count


    # Display the annotated video in output frame
    with video_output:
        clear_output()
        
        # Retrieve the uploaded video content
        if uploaded_video_content:
            try:
                # Save the video content to a temporary file
                with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
                    tmp_file.write(uploaded_video_content)
                    video_path = tmp_file.name

                # Perform object detection on the video
                cap = cv2.VideoCapture(video_path)
                annotated_frames = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                     # Perform object detection on the frame (replace this with your object detection code)
                    ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
                    for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                        if ClassInd <= 80:
                            cv2.rectangle(frame, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (0, 255, 0), 2)
                            cv2.putText(frame, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    
                    annotated_frames.append(frame)

                # Check if frames are present before processing
                if annotated_frames:
                    # Get frame dimensions
                    frame_height, frame_width, _ = annotated_frames[0].shape

                    # Generate the annotated video filename
                    annotated_video_filename = f"annotated_video{annotated_video_count}.mp4"
                    
                    # Convert annotated frames to video format
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    video_writer = cv2.VideoWriter(annotated_video_filename, fourcc, 30, (frame_width, frame_height))
                    
                    for annotated_frame in annotated_frames:
                        video_writer.write(annotated_frame)

                    video_writer.release()

                    # Open the annotated video file
                    annotated_video = open(annotated_video_filename, 'rb')
                
                    # Display annotated video
                    video_widget = widgets.Video(value=annotated_video.read(), format='mp4', width=640, height=480,autoplay=True)
                    display(video_widget)

                else:
                    print("No frames found in the video.")

            except Exception as e:
                print("Error processing video:", e)

        else:
            print("No valid video content available. Please upload a video.")



#Create label
video_label=widgets.HTML(
    value="<h2>Object Detection in VIDEO</h2>"
)

# Create video upload button
video_upload_button = widgets.FileUpload(accept='.mp4', description='Upload Video')


# Create detect video button
detect_video_button = widgets.Button(
    description=' Detect in Video',
    disabled=False,
    button_style='Danger',
    tooltip='Detect Video',
    icon='video' # (FontAwesome names without the `fa-` prefix)
)
detect_video_button.on_click(handle_detect_video_button_click)

# Create loader for video
video_loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Videooooooo...',
    bar_style='danger',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

# Create output widget for video
video_output = widgets.Output(layout={'height': '1000px'})

# Create container widget for video-related components
video_tab_content = widgets.VBox([video_label,video_upload_button, video_loader, widgets.HBox([detect_video_button], layout=widgets.Layout(margin='10px 0px 0px 0px')), video_output])

# Attach event handlers to the video upload button
video_upload_button.observe(handle_video_upload, names='value')

# Create tabs for video-------------------+++++++++++++
tab_contents = ['Video']
tabs = widgets.Tab()
tabs.children = [video_tab_content]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)

# Set height of the entire tab widget
tabs_container = widgets.VBox([tabs], layout={'height': '1000px'})

# Display the tabs
display(tabs_container)


In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image, HTML, Javascript
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
from base64 import b64decode
from base64 import b64encode  
import tempfile
import time
import os


# Load model and labels
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5) # 255/2=127.5
model.setInputMean((127.5, 127.5, 127.5)) # mobilenet => [-1,1]
model.setInputSwapRB(True) # auto convert BGR to RGB

classLabels = []
file_name = 'Labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    # print(classLabels)

# Function to handle image upload--------------
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.01)  # Sleep for 10 milliseconds for smooth progression

             # Update description when progress reaches maximum
            if i == 100:
                loader.description = 'Image Uploaded 100% !!!'  # Change description to 'Uploaded'
                
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content, height='400px', width='600px')
        
        # Display the image
        with uploaded_output:
            clear_output()
            display(img_widget)
             
        # Reset the FileUpload widget to display "Upload(0)"
        image_button.value.clear()


# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with uploaded_output:
        clear_output()
        uploaded_files = image_upload_button.value
        if uploaded_files:
            uploaded_file = uploaded_files[0]  # Access the first uploaded file
            image_content = uploaded_file['content']
            
            image = np.frombuffer(image_content, np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)

            # Perform object detection on the image
            ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
    
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                    cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
            # Convert image to base64 encoding
            _, buffer = cv2.imencode('.jpg', image)
            img_str = base64.b64encode(buffer).decode('utf-8')
    
            # Display annotated image
            display(Image(data=buffer, format='jpg'))
            

# Create buttons for image
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image', multiple=False)

#Create label
# image_label=widgets.HTML(
#     value="<h2>Object Detection in IMAGE (Upload or Capture)</h2>"
# )

# Create detect button
detect_button = button = widgets.Button(
    description='Detect in Uploaded Image',
    disabled=False,
    button_style='Success',
    tooltip='Detect Image',
    icon='image', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='200px')  # Set width to 200px
)

detect_button.on_click(handle_detect_button_click)


#Function to perform object detection in captured images
def handle_detect_captured_button_click(b):
    with captured_output:
        clear_output()
        
        # Capture image from webcam
        cap = cv2.VideoCapture(0)  # 0 for default webcam
        
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            # Display the resulting frame
            cv2.imshow('Capture Image in Webcam(Press Space to Capture Image)', frame)
            
            # Check for 'q' or 'Space' key press
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord(' '):
                # Capture image on spacebar press
                cv2.imwrite("captured_image.jpg", frame)
                print("🎉 Image captured successfully!!! 🎉")
                cap.release()
                cv2.destroyAllWindows()
                break
                
        # Perform object detection on the captured image
        image = cv2.imread("captured_image.jpg")
        ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
        
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                if classLabels:  # Check if class labels are loaded
                    cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Convert image to base64 for display in captured_output widget
        _, buffer = cv2.imencode('.jpg', image)
        img_str = b64encode(buffer).decode('utf-8')
        
        # Display annotated image in captured_output widget
        # captured_img_widget = widgets.Image(value=image_content, height='400px', width='600px')
        # display(img_widget)

        # Display annotated image
        display(Image(data=buffer, format='jpg'))

# Create detect button
detect_capture_button = button = widgets.Button(
    description='Detect in Captured Image',
    disabled=False,
    button_style='Info',
    tooltip='Detect Image',
    icon='camera', 
    layout=widgets.Layout(width='200px')  # Set width to 200px
)

# Attach event handlers to the image upload button
image_upload_button.observe(handle_image_upload, names='value')

detect_capture_button.on_click(handle_detect_captured_button_click)



# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Image...',
    bar_style='success',
    style={'bar_color': 'red', 'description_width': 'initial'},
    orientation='horizontal',
    format='<font color="black">%(value)d%%</font>'  # Show progress percentage in black font color
)

# Create output widget for UPLOADED image
uploaded_output = widgets.Output(layout={'height': '400px', 'width': '600px'})

# Create output widget for CAPTURED Image
captured_output = widgets.Output(layout={'height': '400px', 'width': '600px'})


# Create container widget for image-related components
uploaded_image_tab_content = widgets.VBox([image_upload_button,loader, widgets.HBox([detect_button], layout=widgets.Layout(margin='10px 10px 0px 0px')), uploaded_output])

# Create container widget for image-related components
captured_image_tab_content = widgets.VBox([widgets.HBox([detect_capture_button], layout=widgets.Layout(margin='10px 10px 0px 0px')), captured_output])

image_tab_content = widgets.Accordion(children=[uploaded_image_tab_content, captured_image_tab_content], titles=('1. Object Detection in UPLOADED Image', '2. Object Detection in CAPTURED Image'))


# VIDEO DETECTION--------------------------------
# Define global variables
uploaded_video_content = None

# Function to handle VIDEO upload
def handle_video_upload(change):
    global uploaded_video_content

    if len(change['new']) > 0:
        uploaded_file = change['new'][0]  # Access the first element of the tuple
        uploaded_video_content = uploaded_file['content']
            
        # Simulate progress with sleep
        for i in range(101):
            video_loader.value = i
            time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression

            # Update description when progress reaches maximum
            if i == 100:
                video_loader.description = 'Video Uploaded 100% !!!'  # Change description to 'Uploaded'
                

# Function to handle detect video button click
def handle_detect_video_button_click(b):
    global uploaded_video_content

    if uploaded_video_content:
        # Save the video content to a temporary file
        with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
            tmp_file.write(uploaded_video_content)
            video_path = tmp_file.name

        # Create VideoCapture object for uploaded video
        capt = cv2.VideoCapture(video_path)

        #check if video opened correctly or not
        if not capt.isOpened():
            raise IOError("Cannot open Video") #replace with webcam for webcam detection
        
        font_scale= 3
        font= cv2.FONT_HERSHEY_PLAIN
        
        while True:
            ret, frame= capt.read()
           
            ClassIndex, confidence, bbox = model.detect(frame,confThreshold=0.55) #0.5= 50%
        
            print(ClassIndex)
            if(len(ClassIndex)!=0):
                    for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(),bbox): #all 3 variable in zip
                        if(ClassInd <=80):
                            cv2.rectangle(frame, boxes,(255,0,0),2) #red color rectangle adn text is green
                            cv2.putText(frame, classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font, fontScale=font_scale, color=(0,255,0), thickness=3)
            
            cv2.imshow("Object Detection in Video", frame)
        
            if cv2.waitKey(2) & 0xff==ord('q'):
                break
        
        capt.release()
        cv2.destroyAllWindows()


#Create label
video_label=widgets.HTML(
    value="<h2>Object Detection in VIDEO</h2>"
)

# Create video upload button
video_upload_button = widgets.FileUpload(accept='.mp4', description='Upload Video')


# Create detect video button
detect_video_button = widgets.Button(
    description=' Detect in Video',
    disabled=False,
    button_style='Danger',
    tooltip='Detect Video',
    icon='video' # (FontAwesome names without the `fa-` prefix)
)
detect_video_button.on_click(handle_detect_video_button_click)

# Create loader for video
video_loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading Video...',
    bar_style='danger',
    style={'bar_color': '#ffff00', 'description_width': 'initial'},
    orientation='horizontal',
    format='<font color="black">%(value)d%%</font>'  # Show progress percentage in black font color
)


# Create container widget for video-related components
video_tab_content = widgets.VBox([video_label,video_upload_button, video_loader, widgets.HBox([detect_video_button], layout=widgets.Layout(margin='10px 0px 0px 0px'))])

# Attach event handlers to the video upload button
video_upload_button.observe(handle_video_upload, names='value')


# Webcam Detection--------------------------------
# Create label for webcam detection
webcam_label = widgets.HTML(value="<h2>Webcam Detection</h2>")

# Create button to open webcam
open_webcam_button = widgets.Button(
    description="Open Webcam",
      button_style='warning',
    tooltip='Open Webcam',
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)

# Function to open webcam
def open_webcam(b):
    # Create VideoCapture object for webcam
    capt = cv2.VideoCapture(0)  # Open default webcam (change to 1 if needed)
    
    # Check if webcam opened correctly
    if not capt.isOpened():
        raise IOError("Cannot open Webcam")
    
    # Function to display webcam feed with object detection
    def display_webcam_with_detection():
        ret, frame = capt.read()
        if ret:
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)  # Perform object detection
            if len(ClassIndex) != 0:
                for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                    if ClassInd <= 80:
                        cv2.rectangle(frame, boxes, (255, 0, 0), 2)  # Draw rectangle around detected object
                        cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)  # Display class label
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cv2.imshow("Object Detection in Webcam", frame)  # Display webcam feed with object detection
        
        # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):  # Check if 'q' key is pressed
            cv2.destroyAllWindows()  # Close OpenCV windows
            capt.release()  # Release VideoCapture object
            return
    
    # Display webcam feed with object detection
    display_webcam_with_detection()
    
    # Update webcam feed continuously with object detection
    while capt.isOpened():
        display_webcam_with_detection()
    
# Attach event handlers to the buttons
open_webcam_button.on_click(open_webcam)

# Define a container for the webcam label, open webcam button, close webcam button, and webcam output
webcam_controls = widgets.VBox([webcam_label,open_webcam_button])


# LOGOUT SECTION--------------------------------
# Create label for Logout
logout_label = widgets.HTML(value="<h2><span style='font-size: 24px;'>&#x1F44B;</span> See You Again!!! <span style='font-size: 24px;'>&#x1F44B;</span></h2>")

# Create button to logout
logout_button = widgets.Button(
    description="Logout",
      button_style='danger',
    tooltip='Logout',
    icon='sign-out' # (FontAwesome names without the `fa-` prefix)
)

# Function to logout
def logout_function(b):
    # print("Logout! 😀")  # Emoji for welcome 
    # time.sleep(3)
    
    # Clear username and password text boxes
    username_textbox.value = ''
    password_textbox.value = ''

    # Clear uploaded image and video content
    uploaded_image_content = None
    uploaded_video_content = None

    # Reset file upload widgets
    image_upload_button.value = ()
    video_upload_button.value = ()
    
    # Clear login status output
    with login_status_output:
        clear_output()

    # Show the login tab and hide other tabs
    login_ui.layout.display = 'block'
    tabs_container.layout.display = 'none'
    
# Attach event handlers to the buttons
logout_button.on_click(logout_function)

# Define a container for the logout button
logout_content = widgets.VBox([logout_label,logout_button])


# Function to authenticate login++++++++++++++++++++++++++++++++
def authenticate_login(username, password):
    # Replace this with your actual login authentication logic
    if username == "admin" and password == "admin":
        return True
    else:
        return False

# Username and password text boxes
username_textbox = widgets.Text(placeholder="Enter username")
password_textbox = widgets.Password(placeholder="Enter password")

# Login button
login_button = widgets.Button(description="Login", button_style="success")

# Output widget for displaying login status
login_status_output = widgets.Output()

# Function to handle login button click
def handle_login_button_click(b):
    with login_status_output:
        clear_output()
        username = username_textbox.value.strip()
        password = password_textbox.value.strip()
        
        if authenticate_login(username, password):
            print("😀 Login Successful! 😀")
            time.sleep(2)
         
           
            # Show other tabs and hide the login tab 
            login_ui.layout.display = 'none'
            tabs_container.layout.display = 'block'
            
        else:
            print("Invalid username or password. Please try again.")

# Attach event handler to the login button
login_button.on_click(handle_login_button_click)

# Arrange widgets in a VBox layout
login_ui = widgets.VBox([
    widgets.HTML(value="<h3>Welcome to Login Panel</h3>"),
    widgets.HBox([widgets.Label(value="Username:"), username_textbox]),
    widgets.HBox([widgets.Label(value="Password :"), password_textbox]),
    login_button,
    login_status_output
])

# Create tabs for image, video, and webcam-------------------+++++++++++++
tab_contents = ['Image', 'Video','Webcam','Logout']
tabs = widgets.Tab()
tabs.children = [image_tab_content, video_tab_content,webcam_controls,logout_content]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)


# Set height of the entire tab widget
tabs_container = widgets.VBox([tabs], layout={'height': '1000px'})

# Display the login UI
display(login_ui)

# Display the tabs
display(tabs_container)

# Initially, only show the login tab
tabs_container.layout.display = 'none'

# Set the height of the output cell
display(HTML("<style>.output { height: 1200px; }</style>"))